# Notes:
- Code for analysis under section "Cortisol changes which edges are involved in predictive networks "; result in Table S2-4
- generate data for plot Figure3 and Figure S3

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import combinations
total_edges=70876


# Functions

In [2]:
from scipy.special import comb

def hypergeom_pmf(N, A, n, x):
    
    '''
    Probability Mass Function for Hypergeometric Distribution
    :param N: population size
    :param A: total number of desired items in N
    :param n: number of draws made from N
    :param x: number of desired items in our draw of n items
    :returns: PMF computed at x
    '''
    Achoosex = comb(A,x,exact=True)
    NAchoosenx = comb(N-A, n-x,exact=True)
    Nchoosen = comb(N,n,exact=True)
    
    return (Achoosex)*NAchoosenx/Nchoosen


def hypergeom_cdf(N, A, n, t, min_value=None):
    
    '''
    Cumulative Density Funtion for Hypergeometric Distribution
    :param N: population size
    :param A: total number of desired items in N
    :param n: number of draws made from N
    :param t: number of desired items in our draw of n items up to t
    :returns: CDF computed up to t
    '''
    if min_value:
        return np.sum([hypergeom_pmf(N, A, n, x) for x in range(min_value, t+1)])
    
    return np.sum([hypergeom_pmf(N, A, n, x) for x in range(t+1)])


In [3]:

from scipy.stats import hypergeom

def fast_hypergeom_pmf(N, A, n, x):
    return hypergeom.pmf(x, N, A, n)

def fast_hypergeom_cdf(N, A, n, t, min_value=None):
    if min_value is not None:
        return hypergeom.cdf(t, N, A, n) - hypergeom.cdf(min_value - 1, N, A, n)
    return hypergeom.cdf(t, N, A, n)



In [4]:
def get_overlap(n1,n2,network_ls):
    overlap=np.sum(network_ls[n1].astype(int) &network_ls[n2].astype(int))/2
    n1_edges=np.sum(network_ls[n1])/2
    n2_edges=np.sum(network_ls[n2])/2
    return overlap,n1_edges,n2_edges

In [5]:
def get_overlap_across_constr(n1,n2,network_ls1,network_ls2):
    overlap=np.sum(network_ls1[n1].astype(int) &network_ls2[n2].astype(int))/2
    n1_edges=np.sum(network_ls1[n1])/2
    n2_edges=np.sum(network_ls2[n2])/2
    return overlap,n1_edges,n2_edges

In [6]:
def load_network_data(pheno, folder, p='0.01', id_by='_byTrial'):
   """
   Load network data from files based on phenotype.
   
   Parameters
   ----------
   pheno : str
       Phenotype name (e.g., 'RecogAcc_coded')
   folder : str
       Path to folder containing network files
   p : str, optional
       P-value threshold, default='0.01'  
   id_by : str, optional
       Identifier suffix, default='_byTrial'
   
   Returns
   -------
   dict
       Dictionary containing network data for each condition
   """
   cond_ls = ['cortisol', 'placebo']
   stim_ls = ['alc', 'tool'] 
   direction_ls = ['pos', 'neg']
   network_ls = {}

   for cond in cond_ls:
       for stim in stim_ls:
           for direction in direction_ls:
               filename = f"{folder}n26_{pheno}{id_by}_{cond}_{stim}_htBPfiltered_p{p}_{direction}.txt"
               data = np.loadtxt(filename, delimiter=',')
               network_ls[f"{cond}_{stim}_{direction}{id_by}"] = data
               
   return network_ls

In [7]:
def calculate_network_overlaps(network_ls, total_edges, pheno):
    """
    Calculate overlap statistics between memory predictive networks.
    
    Parameters:
    -----------
    network_ls : dict
        Dictionary containing network data for different conditions
        Expected format: {'condition_name': network_data, ...}
    total_edges : int
        Total number of edges in the complete network
        
    Returns:
    --------
    tuple
        Contains three 8x8 matrices (numpy arrays):
        - overlap_edges: Number of overlapping edges between each pair of networks
        - overlap_percent: Percentage of overlap relative to total edges in both networks
        - overlap_pvalues: P-values from hypergeometric test for significance of overlap
    """
    n_networks = len(network_ls)
    network_names = list(network_ls.keys())
    
    # Initialize matrices to store results
    overlap_edges = np.zeros([n_networks, n_networks])
    overlap_percent = np.zeros([n_networks, n_networks])
    overlap_pvalues = np.zeros([n_networks, n_networks])
    
    # Calculate overlaps for each pair of networks
    for i, n1 in enumerate(network_names):
        for j, n2 in enumerate(network_names[i+1:], i+1):
            # Calculate overlap statistics
            overlap, n1_edges, n2_edges = get_overlap(n1, n2, network_ls)
            p_value = 1 - fast_hypergeom_cdf(total_edges, n1_edges, n2_edges, int(overlap))
            overlap_percent_val = overlap / (n1_edges + n2_edges - overlap)
            
            # Store results in matrices (both directions due to symmetry)
            overlap_edges[i, j] = overlap_edges[j, i] = overlap
            overlap_percent[i, j] = overlap_percent[j, i] = overlap_percent_val
            overlap_pvalues[i, j] = overlap_pvalues[j, i] = p_value if overlap > 0 else np.nan
            
            # Print results only for networks with same direction (pos/neg) and condition (alc/tool)
            same_direction = ('pos' in n1 and 'pos' in n2) or ('neg' in n1 and 'neg' in n2)
            same_condition = ('alc' in n1 and 'alc' in n2) or ('tool' in n1 and 'tool' in n2)
            same_pill = ('cortisol' in n1 and 'cortisol' in n2) or ('placebo' in n1 and 'placebo' in n2)

            if same_direction and (same_condition or same_pill):
                print(f"Network comparison: {pheno}_{n1.strip('_byTrial')} vs {pheno}_{n2.strip('_byTrial')}")
                print(f"Number of edges: {n1_edges} vs {n2_edges}")
                print(f"Overlap edges: {overlap}")
                print(f"HCDF - p: {p_value:.3f}\n")
    
    return overlap_edges, overlap_percent, overlap_pvalues

In [8]:
def save_network_overlaps(network_ls1, network_ls2=None, pheno="", output_folder="results/network_overlap/"):
    """
    Calculate and save network overlaps for networks with same direction and emotionality.
    Can compare networks within a single list or between two different lists.
    
    Parameters
    ----------
    network_ls1 : dict
        Dictionary containing first set of network data
    network_ls2 : dict, optional
        Dictionary containing second set of network data
        If None, will compare networks within network_ls1
        Default: None
    pheno : str
        Phenotype name used in output filenames
    output_folder : str, optional
        Base folder path for saving overlap files
        Default: 'data/5s_onward/network_overlap/'
        
    Returns
    -------
    None
        Saves overlap files to disk for matching network pairs
    """
    # Create output subfolder
    save_folder = f"{output_folder}/"
    
    if network_ls2 is None:
        # Original behavior - compare within single list
        network_pairs = list(combinations(network_ls1.keys(), 2))
        
        # Calculate and save overlap for matching pairs
        for pair in network_pairs:
            n1, n2 = pair[0], pair[1]
            
            # Check if networks have same direction and condition
            same_direction = ('pos' in n1 and 'pos' in n2) or ('neg' in n1 and 'neg' in n2)
            same_condition = ('cortisol' in n1 and 'cortisol' in n2) or ('placebo' in n1 and 'placebo' in n2)
            
            if same_direction and same_condition:
                # Calculate overlap
                overlap = network_ls1[n1].astype(int) & network_ls1[n2].astype(int)
                overlap = overlap / 2
                
                # Create filename - remove '_byTrial' suffix
                filename = (f"{save_folder}{pheno}_{n1[:-8]}&{n2[:-8]}"
                          "_overlap.txt")
                
                # Save to file
                np.savetxt(filename, overlap, delimiter=',')
    
    else:
        # Compare between two different lists
        for n1 in network_ls1.keys():
            for n2 in network_ls2.keys():
                # Check if networks have same direction and condition
                same_direction = ('pos' in n1 and 'pos' in n2) or ('neg' in n1 and 'neg' in n2)
                same_condition = ('cortisol' in n1 and 'cortisol' in n2) or ('placebo' in n1 and 'placebo' in n2)
                
                if same_direction and same_condition:
                    # Calculate overlap
                    overlap = network_ls1[n1].astype(int) & network_ls2[n2].astype(int)
                    overlap = overlap / 2
                    
                    # Create filename with memory and arousal prefixes
                    filename = (f"{save_folder}memory_{n1[:-8]}&arousal_{n2[:-8]}"
                              "_overlap.txt")
                    
                    # Save to file
                    np.savetxt(filename, overlap, delimiter=',')

In [9]:
def chi_output(data):
    stat, p, dof, expected = chi2_contingency(data)
    alpha = 0.05

    print("p value is " + "{:.3f}".format(p))
    print("stat " + "{:.3f}".format(stat))
    """ 
    # other outputs
    print("expected:",expected)
    if p <= alpha:
        print('Dependent (reject H0)')
    else:
        print('Independent (H0 holds true)')
    """
            
def chi_sq(n1,n2,n3,n4,network_ls,sep=False,pheno="memory"):
    print('Chi sq Test: \n')
    if len(network_ls)==1:
        overlap1,n1_edges,n2_edges= get_overlap(n1,n2,network_ls[0])
        overlap2, n3_edges,n4_edges=get_overlap(n3,n4,network_ls[0])
        pheno1 = pheno
        pheno2 = pheno
    else:
        pheno1 = 'memory'
        pheno2 = 'arousal' 
        overlap1,n1_edges,n2_edges=get_overlap_across_constr(n1,n2,network_ls[0],network_ls[1])
        overlap2, n3_edges,n4_edges=get_overlap_across_constr(n3,n4,network_ls[0],network_ls[1])

    print(f"Overlap1: {pheno1}_{n1.strip('_byTrial')} vs {pheno2}_{n2.strip('_byTrial')}")
    print(f"Number of edges: {n1_edges} vs {n2_edges}, Overlap edges: {overlap1}")

    print(f"Overlap2: {pheno1}_{n3.strip('_byTrial')} vs {pheno2}_{n4.strip('_byTrial')}")
    print(f"Number of edges: {n3_edges} vs {n4_edges}, Overlap edges: {overlap2}")


    if (sep):
        #defining the table
        data = [[overlap1, overlap2], [ n1_edges-overlap1, n3_edges-overlap2]]
        print('n: '+str(n1_edges+n3_edges))
        chi_output(data)
        
        data = [[overlap1, overlap2], [ n2_edges-overlap1, n4_edges-overlap2]]
        print('n: '+str(n2_edges+n4_edges))
        chi_output(data)


    else:
        data = [[overlap1, overlap2], [ n1_edges+n2_edges-overlap1*2, n3_edges+n4_edges-overlap2*2]]
        print('n: '+str(np.sum(data)))

        chi_output(data)

In [10]:
folder="CPM code/result/network_edges/"
id_by='_byTrial'

# Item memory networks overlap

In [11]:
network_ls_recog=load_network_data("RecogAcc_coded", folder)
save_network_overlaps(network_ls_recog,pheno="memory")
edges_matrix, percent_matrix, pvalue_matrix = calculate_network_overlaps(network_ls_recog, total_edges,'memory')
np.savetxt("results/network_overlap/memory_network_overlap_percent.txt",percent_matrix,delimiter=',')

Network comparison: memory_cortisol_alc_pos vs memory_cortisol_tool_pos
Number of edges: 521.0 vs 582.0
Overlap edges: 25.0
HCDF - p: 0.000

Network comparison: memory_cortisol_alc_pos vs memory_placebo_alc_pos
Number of edges: 521.0 vs 192.0
Overlap edges: 3.0
HCDF - p: 0.054

Network comparison: memory_cortisol_alc_neg vs memory_cortisol_tool_neg
Number of edges: 111.0 vs 371.0
Overlap edges: 1.0
HCDF - p: 0.115

Network comparison: memory_cortisol_alc_neg vs memory_placebo_alc_neg
Number of edges: 111.0 vs 217.0
Overlap edges: 1.0
HCDF - p: 0.046

Network comparison: memory_cortisol_tool_pos vs memory_placebo_tool_pos
Number of edges: 582.0 vs 515.0
Overlap edges: 20.0
HCDF - p: 0.000

Network comparison: memory_cortisol_tool_neg vs memory_placebo_tool_neg
Number of edges: 371.0 vs 2423.0
Overlap edges: 86.0
HCDF - p: 0.000

Network comparison: memory_placebo_alc_pos vs memory_placebo_tool_pos
Number of edges: 192.0 vs 515.0
Overlap edges: 9.0
HCDF - p: 0.000

Network comparison: me

In [12]:
n1='cortisol_alc_pos_byTrial'
n2='cortisol_tool_pos_byTrial'
n3='placebo_alc_pos_byTrial'
n4='placebo_tool_pos_byTrial'
chi_sq(n1,n2,n3,n4,[network_ls_recog],pheno='memory')

Chi sq Test: 

Overlap1: memory_cortisol_alc_pos vs memory_cortisol_tool_pos
Number of edges: 521.0 vs 582.0, Overlap edges: 25.0
Overlap2: memory_placebo_alc_pos vs memory_placebo_tool_pos
Number of edges: 192.0 vs 515.0, Overlap edges: 9.0
n: 1776.0
p value is 0.171
stat 1.875


In [13]:
n1='cortisol_alc_neg_byTrial'
n2='cortisol_tool_neg_byTrial'
n3='placebo_alc_neg_byTrial'
n4='placebo_tool_neg_byTrial'
chi_sq(n1,n2,n3,n4,[network_ls_recog],pheno='memory')

Chi sq Test: 

Overlap1: memory_cortisol_alc_neg vs memory_cortisol_tool_neg
Number of edges: 111.0 vs 371.0, Overlap edges: 1.0
Overlap2: memory_placebo_alc_neg vs memory_placebo_tool_neg
Number of edges: 217.0 vs 2423.0, Overlap edges: 62.0
n: 3059.0
p value is 0.003
stat 8.642


# Arousal overlap


In [14]:
network_ls_arous=load_network_data("Arous_All", folder)
save_network_overlaps(network_ls_arous,pheno="arousal")
edges_matrix, percent_matrix, pvalue_matrix = calculate_network_overlaps(network_ls_arous, total_edges,'arousal')
np.savetxt("results/network_overlap/arousal_network_overlap_percent.txt",percent_matrix,delimiter=',')

Network comparison: arousal_cortisol_alc_pos vs arousal_cortisol_tool_pos
Number of edges: 3873.0 vs 2180.0
Overlap edges: 702.0
HCDF - p: 0.000

Network comparison: arousal_cortisol_alc_pos vs arousal_placebo_alc_pos
Number of edges: 3873.0 vs 1279.0
Overlap edges: 162.0
HCDF - p: 0.000

Network comparison: arousal_cortisol_alc_neg vs arousal_cortisol_tool_neg
Number of edges: 796.0 vs 3099.0
Overlap edges: 350.0
HCDF - p: 0.000

Network comparison: arousal_cortisol_alc_neg vs arousal_placebo_alc_neg
Number of edges: 796.0 vs 537.0
Overlap edges: 19.0
HCDF - p: 0.000

Network comparison: arousal_cortisol_tool_pos vs arousal_placebo_tool_pos
Number of edges: 2180.0 vs 854.0
Overlap edges: 116.0
HCDF - p: 0.000

Network comparison: arousal_cortisol_tool_neg vs arousal_placebo_tool_neg
Number of edges: 3099.0 vs 2154.0
Overlap edges: 447.0
HCDF - p: 0.000

Network comparison: arousal_placebo_alc_pos vs arousal_placebo_tool_pos
Number of edges: 1279.0 vs 854.0
Overlap edges: 123.0
HCDF - 

In [15]:
n1='cortisol_alc_pos_byTrial'
n2='cortisol_tool_pos_byTrial'
n3='placebo_alc_pos_byTrial'
n4='placebo_tool_pos_byTrial'
chi_sq(n1,n2,n3,n4,[network_ls_arous],pheno='arousal')

Chi sq Test: 

Overlap1: arousal_cortisol_alc_pos vs arousal_cortisol_tool_pos
Number of edges: 3873.0 vs 2180.0, Overlap edges: 702.0
Overlap2: arousal_placebo_alc_pos vs arousal_placebo_tool_pos
Number of edges: 1279.0 vs 854.0, Overlap edges: 123.0
n: 7361.0
p value is 0.000
stat 71.236


In [16]:
n1='cortisol_alc_neg_byTrial'
n2='cortisol_tool_neg_byTrial'
n3='placebo_alc_neg_byTrial'
n4='placebo_tool_neg_byTrial'
chi_sq(n1,n2,n3,n4,[network_ls_arous],pheno='arousal')

Chi sq Test: 

Overlap1: arousal_cortisol_alc_neg vs arousal_cortisol_tool_neg
Number of edges: 796.0 vs 3099.0, Overlap edges: 350.0
Overlap2: arousal_placebo_alc_neg vs arousal_placebo_tool_neg
Number of edges: 537.0 vs 2154.0, Overlap edges: 107.0
n: 6129.0
p value is 0.000
stat 70.340


# Arousal and Item_mem overlap

In [17]:
save_network_overlaps(network_ls_recog,network_ls_arous)

In [18]:
total_edges=70876
overlap_e = np.zeros([8, 8])
overlap_p = np.zeros([8, 8])
overlap_e_percent=np.zeros([8, 8])
for n1 in network_ls_recog:
    for n2 in network_ls_arous:
        n1_index=list(network_ls_recog.keys()).index(n1)
        n2_index=list(network_ls_arous.keys()).index(n2)
        
        overlap,n1_edges,n2_edges= get_overlap_across_constr(n1,n2,network_ls_recog,network_ls_arous)
        
        p=1-fast_hypergeom_cdf(total_edges,n1_edges,n2_edges,int(overlap))
        overlap_e[n1_index,n2_index]=overlap
        overlap_e_percent[n1_index,n2_index]=overlap/(n1_edges+n2_edges-overlap)

        
        if overlap==0:
            overlap_p[n1_index,n2_index]=np.nan
        else:
            overlap_p[n1_index,n2_index]=p
        same_direction = ('pos' in n1 and 'pos' in n2) or ('neg' in n1 and 'neg' in n2)
        same_condition = ('cortisol' in n1 and 'cortisol' in n2) or ('placebo' in n1 and 'placebo' in n2)

        if same_direction and same_condition:
            print(f"Network comparison: memory_{n1.strip('_byTrial')} vs arousal_{n2.strip('_byTrial')}")
            print(f"Number of edges: {n1_edges} vs {n2_edges}")
            print(f"Overlap edges: {overlap}")
            print(f"HCDF - p: {p:.3f}\n")


Network comparison: memory_cortisol_alc_pos vs arousal_cortisol_alc_pos
Number of edges: 521.0 vs 3873.0
Overlap edges: 97.0
HCDF - p: 0.000

Network comparison: memory_cortisol_alc_pos vs arousal_cortisol_tool_pos
Number of edges: 521.0 vs 2180.0
Overlap edges: 62.0
HCDF - p: 0.000

Network comparison: memory_cortisol_alc_neg vs arousal_cortisol_alc_neg
Number of edges: 111.0 vs 796.0
Overlap edges: 6.0
HCDF - p: 0.000

Network comparison: memory_cortisol_alc_neg vs arousal_cortisol_tool_neg
Number of edges: 111.0 vs 3099.0
Overlap edges: 4.0
HCDF - p: 0.537

Network comparison: memory_cortisol_tool_pos vs arousal_cortisol_alc_pos
Number of edges: 582.0 vs 3873.0
Overlap edges: 108.0
HCDF - p: 0.000

Network comparison: memory_cortisol_tool_pos vs arousal_cortisol_tool_pos
Number of edges: 582.0 vs 2180.0
Overlap edges: 86.0
HCDF - p: 0.000

Network comparison: memory_cortisol_tool_neg vs arousal_cortisol_alc_neg
Number of edges: 371.0 vs 796.0
Overlap edges: 26.0
HCDF - p: 0.000

Net

In [19]:
np.savetxt("results/network_overlap/memory&arousal_network_overlap_percent.txt",overlap_e_percent,delimiter=',')

In [20]:
n1='cortisol_alc_neg_byTrial' # mem
n2='cortisol_alc_neg_byTrial' # arous
n3='placebo_alc_neg_byTrial'
n4='placebo_alc_neg_byTrial'
chi_sq(n1,n2,n3,n4,[network_ls_recog,network_ls_arous])

Chi sq Test: 

Overlap1: memory_cortisol_alc_neg vs arousal_cortisol_alc_neg
Number of edges: 111.0 vs 796.0, Overlap edges: 6.0
Overlap2: memory_placebo_alc_neg vs arousal_placebo_alc_neg
Number of edges: 217.0 vs 537.0, Overlap edges: 4.0
n: 1651.0
p value is 0.978
stat 0.001


In [21]:
n1='cortisol_alc_neg_byTrial' # mem
n2='cortisol_alc_neg_byTrial' # arous
n3='placebo_alc_neg_byTrial'
n4='placebo_alc_neg_byTrial'
chi_sq(n1,n2,n3,n4,[network_ls_recog,network_ls_arous])

Chi sq Test: 

Overlap1: memory_cortisol_alc_neg vs arousal_cortisol_alc_neg
Number of edges: 111.0 vs 796.0, Overlap edges: 6.0
Overlap2: memory_placebo_alc_neg vs arousal_placebo_alc_neg
Number of edges: 217.0 vs 537.0, Overlap edges: 4.0
n: 1651.0
p value is 0.978
stat 0.001


In [22]:
n1='cortisol_tool_pos_byTrial' # mem
n2='cortisol_tool_pos_byTrial' # aours
n3='placebo_tool_pos_byTrial'
n4='placebo_tool_pos_byTrial'
chi_sq(n1,n2,n3,n4,[network_ls_recog,network_ls_arous])

Chi sq Test: 

Overlap1: memory_cortisol_tool_pos vs arousal_cortisol_tool_pos
Number of edges: 582.0 vs 2180.0, Overlap edges: 86.0
Overlap2: memory_placebo_tool_pos vs arousal_placebo_tool_pos
Number of edges: 515.0 vs 854.0, Overlap edges: 13.0
n: 4032.0
p value is 0.000
stat 18.178


In [23]:
n1='cortisol_tool_neg_byTrial'#mem
n2='cortisol_tool_neg_byTrial'#arous
n3='placebo_tool_neg_byTrial'
n4='placebo_tool_neg_byTrial'
chi_sq(n1,n2,n3,n4,[network_ls_recog,network_ls_arous])

Chi sq Test: 

Overlap1: memory_cortisol_tool_neg vs arousal_cortisol_tool_neg
Number of edges: 371.0 vs 3099.0, Overlap edges: 80.0
Overlap2: memory_placebo_tool_neg vs arousal_placebo_tool_neg
Number of edges: 2423.0 vs 2154.0, Overlap edges: 212.0
n: 7755.0
p value is 0.000
stat 32.145


In [24]:
n1='cortisol_alc_pos_byTrial' #mem
n2='cortisol_tool_pos_byTrial'#arous
n3='placebo_alc_pos_byTrial'
n4='placebo_tool_pos_byTrial'
chi_sq(n1,n2,n3,n4,[network_ls_recog,network_ls_arous])

Chi sq Test: 

Overlap1: memory_cortisol_alc_pos vs arousal_cortisol_tool_pos
Number of edges: 521.0 vs 2180.0, Overlap edges: 62.0
Overlap2: memory_placebo_alc_pos vs arousal_placebo_tool_pos
Number of edges: 192.0 vs 854.0, Overlap edges: 7.0
n: 3678.0
p value is 0.001
stat 10.479


In [25]:

n1='cortisol_alc_neg_byTrial' #mem
n2='cortisol_tool_neg_byTrial' #arous
n3='placebo_alc_neg_byTrial'
n4='placebo_tool_neg_byTrial'
chi_sq(n1,n2,n3,n4,[network_ls_recog,network_ls_arous])

Chi sq Test: 

Overlap1: memory_cortisol_alc_neg vs arousal_cortisol_tool_neg
Number of edges: 111.0 vs 3099.0, Overlap edges: 4.0
Overlap2: memory_placebo_alc_neg vs arousal_placebo_tool_neg
Number of edges: 217.0 vs 2154.0, Overlap edges: 33.0
n: 5544.0
p value is 0.000
stat 31.852


In [26]:
n1='cortisol_tool_pos_byTrial'#mem
n2='cortisol_alc_pos_byTrial'#arous
n3='placebo_tool_pos_byTrial'
n4='placebo_alc_pos_byTrial'
chi_sq(n1,n2,n3,n4,[network_ls_recog,network_ls_arous])

Chi sq Test: 

Overlap1: memory_cortisol_tool_pos vs arousal_cortisol_alc_pos
Number of edges: 582.0 vs 3873.0, Overlap edges: 108.0
Overlap2: memory_placebo_tool_pos vs arousal_placebo_alc_pos
Number of edges: 515.0 vs 1279.0, Overlap edges: 3.0
n: 6138.0
p value is 0.000
stat 37.053


In [27]:
n1='cortisol_tool_neg_byTrial'#mem
n2='cortisol_alc_neg_byTrial'#arous
n3='placebo_tool_neg_byTrial'
n4='placebo_alc_neg_byTrial'
chi_sq(n1,n2,n3,n4,[network_ls_recog,network_ls_arous])

Chi sq Test: 

Overlap1: memory_cortisol_tool_neg vs arousal_cortisol_alc_neg
Number of edges: 371.0 vs 796.0, Overlap edges: 26.0
Overlap2: memory_placebo_tool_neg vs arousal_placebo_alc_neg
Number of edges: 2423.0 vs 537.0, Overlap edges: 17.0
n: 4084.0
p value is 0.000
stat 21.234
